In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import PIL # must do this before importing lxml because of bug in Pillow/lxml (gzip conflict)
import numpy as np
import tempfile
from itertools import islice
import xml.etree.ElementTree as ET
from lxml import etree
import io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import imageio.v2 as imageio
import subprocess
import os
import webdataset as wds
import glob
import textcheck
import scipy.ndimage as ndi

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual
import ipywidgets as widgets

In [3]:

from importlib import reload
import iautils
reload(iautils)

<module 'iautils' from '/home/tmb/ocropus4extract/iautils.py'>

In [4]:
!gsutil ls gs://ocro-iaa/books | shardsum

gs://ocro-iaa/books/books-{000000..000703}.tar


In [5]:
ds = wds.WebDataset("gs://ocro-iaa/books/books-000000.tar").decode()
sample = next(iter(ds))
print(sample.keys())

dict_keys(['__key__', '__url__', 'abbyy.gz', 'json', 'zip'])


In [6]:
abbyy = sample["abbyy.gz"]
print(abbyy[:500])
print(abbyy[-100:])

b'<?xml version="1.0" encoding="UTF-8"?>\n<document version="1.0" producer="LuraDocument XML Exporter for ABBYY FineReader" pagesCount="298"           \n xmlns="http://www.abbyy.com/FineReader_xml/FineReader6-schema-v1.xml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.abbyy.com/FineReader_xml/FineReader6-schema-v1.xml http://www.abbyy.com/FineReader_xml/FineReader6-schema-v1.xml">\n<page width="1000" height="1500" resolution="500" originalCoords="true">\n</page>'
b'/page>\n<page width="1000" height="1500" resolution="500" originalCoords="true">\n</page>\n</document>\n'


In [45]:
def compute_bounding_box(char_elements):
    lefts = [int(char.get("l")) for char in char_elements]
    tops = [int(char.get("t")) for char in char_elements]
    rights = [int(char.get("r")) for char in char_elements]
    bottoms = [int(char.get("b")) for char in char_elements]

    return min(lefts), min(tops), max(rights), max(bottoms)

def process_line_words(line, trim_spaces=True, abbyy_ns=""):
    # abbyy_ns = "{http://www.abbyy.com/FineReader_xml/FineReader6-schema-v1.xml}"
    new_line = ET.Element("line", attrib=line.attrib)

    for element in line:
        if element.tag == f"{abbyy_ns}word":
            # Trim spaces at the beginning and end of the word if required
            if trim_spaces:
                char_elements = list(element.iter(f"{abbyy_ns}charParams"))
                if char_elements[0].text.isspace():
                    element.remove(char_elements[0])
                    char_elements = char_elements[1:]

                if char_elements[-1].text.isspace():
                    element.remove(char_elements[-1])
                    char_elements = char_elements[:-1]

            # Compute bounding box and update the word's attributes
            l, t, r, b = compute_bounding_box(char_elements)
            element.attrib.update({"l": str(l), "t": str(t), "r": str(r), "b": str(b)})

        new_line.append(element)

    return new_line

In [46]:
import xml.etree.ElementTree as ET

def group_chars_into_words(line, keep_spaces=False):
    abbyy_ns = "{http://www.abbyy.com/FineReader_xml/FineReader6-schema-v1.xml}"
    words = list(line.iter(f"{abbyy_ns}word"))

    # If there are no word elements in the line
    if not words:
        new_line = ET.Element("line", attrib=line.attrib)
        current_word = None

        for char_params in line.iter(f"{abbyy_ns}charParams"):
            char = char_params.text

            if char:
                if char.isspace():  # If the character is a space, create a new word
                    current_word = None
                    if keep_spaces:
                        space_element = ET.SubElement(new_line, "charParams")
                        space_element.text = char
                        space_element.attrib.update(char_params.attrib)
                else:
                    if current_word is None:
                        current_word = ET.Element("word")
                        new_line.append(current_word)

                    char_element = ET.SubElement(current_word, "charParams")
                    char_element.text = char
                    char_element.attrib.update(char_params.attrib)

        return new_line
    else:
        return line

def parse_abbyy_xml(xml_string):
    root = ET.fromstring(xml_string)
    abbyy_ns = "{http://www.abbyy.com/FineReader_xml/FineReader6-schema-v1.xml}"
    new_root = ET.Element("root")

    for page in root.iter(f"{abbyy_ns}page"):
        new_page = ET.Element("page", attrib=page.attrib)
        new_root.append(new_page)

        for block in page.iter(f"{abbyy_ns}block"):
            block_type = block.get("blockType")

            if block_type == "Text":
                for text in block.iter(f"{abbyy_ns}text"):
                    for line in text.iter(f"{abbyy_ns}line"):
                        new_line = group_chars_into_words(line)
                        new_line = process_line_words(new_line)
                        new_page.append(new_line)

            elif block_type in ["Picture", "Table"]:
                new_figure = ET.Element("figure", attrib=block.attrib)
                new_page.append(new_figure)

    return new_root

xml_tree = parse_abbyy_xml(abbyy)

In [47]:
import os
from xml.dom import minidom

def prettify_xml(unformatted_xml):
    if not isinstance(unformatted_xml, str):
        unformatted_xml = ET.tostring(element, encoding="UTF-8")
    parsed_xml = minidom.parseString(unformatted_xml)
    return parsed_xml.toprettyxml(indent="  ")

def write_pages(xml_tree):
    # Iterate over the pages in the XML tree
    for idx, page in enumerate(xml_tree.iter("page")):
        # Create a new root element for the page
        page_root = ET.Element("root")
        page_root.append(page)
        
        stream = io.StringIO()
        ET.ElementTree(page_root).write(stream, encoding="unicode")
        yield idx, prettify_xml(stream.getvalue())

for idx, page in write_pages(xml_tree):
    if idx < 10:
        continue
    print(f"Page {idx}")
    print(page)
    break

Page 10
<?xml version="1.0" ?>
<root>
  <page width="2898" height="4297" resolution="500" originalCoords="true">
    <line baseline="251" l="734" t="183" r="2207" b="257">
      <word l="734" t="183" r="991" b="252">
        <charParams l="734" t="200" r="788" b="249" wordStart="true" wordFromDictionary="true" wordNormal="true" wordNumeric="false" wordIdentifier="false" charConfidence="28" serifProbability="94" wordPenalty="24" meanStrokeWidth="86">H</charParams>
        <charParams l="793" t="199" r="838" b="252" wordStart="false" wordFromDictionary="true" wordNormal="true" wordNumeric="false" wordIdentifier="false" charConfidence="30" serifProbability="100" wordPenalty="24" meanStrokeWidth="86">u</charParams>
        <charParams l="845" t="201" r="887" b="252" wordStart="false" wordFromDictionary="true" wordNormal="true" wordNumeric="false" wordIdentifier="false" charConfidence="26" serifProbability="100" wordPenalty="24" meanStrokeWidth="86">g</charParams>
        <charParams l="892